In [1]:
import pandas as pd

import numpy as np

import pickle

# Load Datasets

In [2]:
#Reference of the dataset from DeepGO
# ! tar -xvzf  CAFA3_training_data/data12G/data/train.tar.gz

### Load Molecular Function Terms

In [4]:
trainMF_terms=pd.read_pickle('../../../data/DeepGo/train-mf.pkl')

trainMF_terms=trainMF_terms[trainMF_terms['orgs']=='9606']

testMF_terms=pd.read_pickle('../../../data/DeepGo/test-mf.pkl')

testMF_terms=testMF_terms[testMF_terms['orgs']=='9606']

trainMF_terms.columns=['uniprot_ac', 'gos', 'labels', 'ngrams', 'proteins', 'sequences','orgs', 'embeddings']

testMF_terms.columns=['uniprot_ac', 'gos', 'labels', 'ngrams', 'proteins', 'sequences','orgs', 'embeddings']

# map uniprot to String

* STring IDs are required for embedding generation

In [5]:
import sys
sys.path.append('../../../utils/')

In [7]:
from  utils import UniprotID_to_StringId

### Convert Train IDs

In [8]:
dfTrain=UniprotID_to_StringId(trainMF_terms)

Not Matched:  173


/home/ubuntuadmin/anaconda37/lib/python3.7/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


In [9]:
dfTrain.head()

,uniprot_ac,gos,labels,ngrams,proteins,sequences,orgs,embeddings,uniprot_ac_uniprot_id,string_id
0,P31946,"[GO:0019904, GO:0019899, GO:0043488, GO:000582...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4331, 6603, 4049, 976, 3504, 6070, 1398, 3954...",1433B_HUMAN,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...,9606,"[-0.690246, -1.16305, 0.082013, -1.17365, 0.89...",P31946|1433B_HUMAN,9606.ENSP00000361930
1,P62258,"[GO:1901016, GO:0086013, GO:0021762, GO:004698...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...","[4043, 855, 1084, 5663, 1250, 998, 3960, 7194,...",1433E_HUMAN,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...,9606,"[-0.707461, -0.440626, 0.28608, -1.23422, 0.47...",P62258|1433E_HUMAN,9606.ENSP00000264335
2,Q04917,"[GO:0042921, GO:0006713, GO:0019899, GO:001708...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4103, 2055, 1084, 5674, 1470, 5390, 3794, 387...",1433F_HUMAN,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...,9606,"[-0.595342, -0.547682, 0.575142, -0.983365, 1....",Q04917|1433F_HUMAN,9606.ENSP00000248975
3,P61981,"[GO:0000086, GO:0045664, GO:0016020, GO:004816...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4343, 6855, 1084, 5674, 1470, 5398, 3954, 706...",1433G_HUMAN,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...,9606,"[-0.661601, 0.378197, 0.508445, -0.970525, 0.4...",P61981|1433G_HUMAN,9606.ENSP00000306330
4,P31947,"[GO:0001836, GO:0008630, GO:0005829, GO:000716...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4075, 1481, 5616, 310, 6188, 3754, 3069, 5361...",1433S_HUMAN,MERASLIQKAKLAEQAERYEDMAAFMKGAVEKGEELSCEERNLLSV...,9606,"[-0.305037, -0.397102, 0.144294, -0.528591, 0....",P31947|1433S_HUMAN,9606.ENSP00000340989


### Convert Test IDs

In [10]:
dfTest=UniprotID_to_StringId(testMF_terms)

Not Matched:  24


# Load TripletProt Network and generate embeddings

In [11]:
from  utils import generate_tripletProt_embeddings

### Generate Train Embeddings

In [12]:
trainProtein_weights=generate_tripletProt_embeddings(dfTrain)







number of found:  6177


In [13]:
trainProtein_weights.shape

(6350, 64)

#### Generate Test Embeddings

In [14]:
testProtein_weights=generate_tripletProt_embeddings(dfTest)

number of found:  1611


In [15]:
testProtein_weights.shape

(1635, 64)

#### Prepare Input Data

In [16]:
X_train=trainProtein_weights

X_train=np.array(X_train)

In [17]:
Ytrain=dfTrain[['labels']]

Ytrain=Ytrain.labels.apply(pd.Series)

Ytrain.shape

(6350, 589)

In [18]:
X_test=testProtein_weights

X_test=np.array(X_test)

In [19]:
Ytest=dfTest[['labels']]

Ytest=Ytest.labels.apply(pd.Series)

Ytest=np.array(Ytest)

Ytest.shape

(1635, 589)

In [20]:
X_train=np.expand_dims(X_train,axis=-1)

In [21]:
X_test=np.expand_dims(X_test,axis=-1)

In [22]:
num_classes=Ytrain.shape[1]

### Train and Evaluate the model

In [31]:
from sklearn.model_selection import StratifiedKFold

from  model import naive_CNN_classifier

import keras

from sklearn.metrics import label_ranking_average_precision_score

from numpy import arange

from sklearn.metrics import f1_score


In [24]:
random_seed=1311

In [26]:
model=None
model = naive_CNN_classifier(num_classes)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [27]:
history=model.fit(X_train, Ytrain, validation_data=(X_test,Ytest) ,epochs=50, batch_size=16,verbose=0)
YtestPredicted_raw=model.predict(X_test)
avePrec =label_ranking_average_precision_score(Ytest, YtestPredicted_raw) 

In [32]:
results={'treshold':[],'Average Precision':[],'F1 (micro)':[],'F1 (macro)':[],'Method':[]}

In [33]:
for treshold in arange(0.05,0.55,0.05):  
    YtestPredicted=None
    YtestPredicted=YtestPredicted_raw.copy()
    results['treshold'].append(treshold)
    results['Average Precision'].append(avePrec)
    YtestPredicted[YtestPredicted>=treshold]=1
    YtestPredicted[YtestPredicted<treshold]=0
    results['F1 (micro)'].append  (f1_score(Ytest, YtestPredicted, average='micro'))
    results['F1 (macro)'].append  (f1_score(Ytest, YtestPredicted, average='macro'))
    results['Method'].append ('TripletProt')


/home/ubuntuadmin/anaconda37/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [34]:
df_results=pd.DataFrame(results)

In [35]:
df_results.head(50)

,treshold,Average Precision,F1 (micro),F1 (macro),Method
0,0.05,0.517938,0.294614,0.146405,TripletProt
1,0.10,0.517938,0.369824,0.152122,TripletProt
2,0.15,0.517938,0.414294,0.152042,TripletProt
3,0.20,0.517938,0.436066,0.144538,TripletProt
4,0.25,0.517938,0.444813,0.134633,TripletProt
5,0.30,0.517938,0.444904,0.121783,TripletProt
6,0.35,0.517938,0.440755,0.112680,TripletProt
7,0.40,0.517938,0.433427,0.102065,TripletProt
8,0.45,0.517938,0.422976,0.093291,TripletProt
9,0.50,0.517938,0.402643,0.083361,TripletProt


In [2]:
df_results.to_csv('../../../data/_Outputs/Final_TripletProt_MF.csv',index=False)